### Fake News Classifier Using LSTM
Dataset: https://www.kaggle.com/c/fake-news/data#

In [166]:
import pandas as pd

In [167]:
df=pd.read_csv('dataset/fake-news/train.csv')

In [168]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [169]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [170]:
###Drop Nan Values
df=df.dropna()

In [171]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [172]:
## Get the Dependent features
y=df['label']

In [173]:
X.shape

(18285, 4)

In [174]:
y.shape

(18285,)

In [175]:
import tensorflow as tf

In [176]:
tf.__version__

'2.3.0'

In [177]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [178]:
### Vocabulary size
voc_size=5000

#### Onehot Representation

In [179]:
messages = X.copy()

In [180]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [181]:
messages.reset_index(inplace = True)

In [182]:
import nltk
import re
from nltk.corpus import stopwords

In [183]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [184]:
len(messages)

18285

In [185]:
### Dataset Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0 ,len(messages)):
    review = re.sub('[^A-Za-z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
#print(review)

In [186]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [187]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1607, 1362, 2825, 1670, 1874, 2628, 3771, 2426, 1426, 4436],
 [1893, 4164, 1268, 2216, 1203, 1938, 1416],
 [2789, 2033, 2661, 3557],
 [4944, 4470, 1571, 3444, 2208, 4371],
 [2817, 1203, 746, 540, 1735, 2115, 1203, 965, 2334, 4426],
 [3211,
  1331,
  3438,
  1845,
  4826,
  49,
  2149,
  3347,
  3053,
  1444,
  1985,
  2699,
  2267,
  4193,
  1416],
 [711, 4858, 300, 2737, 3031, 1257, 976, 2681, 3334, 1972, 1712],
 [1725, 1491, 3800, 2148, 1874, 2505, 49, 2275, 3334, 1972, 1712],
 [478, 813, 257, 3037, 1112, 1422, 2124, 105, 49, 3093],
 [761, 3134, 2914, 71, 2467, 2701, 4429, 1335],
 [79, 4614, 4327, 2198, 4540, 1693, 4124, 1210, 2042, 3907, 58],
 [3444, 1080, 1874, 1422, 49, 1874],
 [4517, 3687, 4279, 4091, 3259, 2680, 1072, 3266, 4898],
 [308, 4237, 757, 2582, 1428, 2143, 4002, 3334, 1972, 1712],
 [2806, 4181, 1665, 878, 838, 3334, 1972, 1712],
 [969, 3588, 1223, 3448, 261, 2351, 3251, 2138, 3435, 4448],
 [2536, 1151, 4164],
 [1532, 2227, 2037, 3812, 49, 4425, 480, 1416],
 [1896, 13

### Embedding Representation

In [188]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen = sent_length)
print(embedded_docs)
len(embedded_docs)

[[   0    0    0 ... 2426 1426 4436]
 [   0    0    0 ... 1203 1938 1416]
 [   0    0    0 ... 2033 2661 3557]
 ...
 [   0    0    0 ... 3334 1972 1712]
 [   0    0    0 ... 1549 4074 1070]
 [   0    0    0 ... 1408  691 4914]]


18285

In [189]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1607,
       1362, 2825, 1670, 1874, 2628, 3771, 2426, 1426, 4436])

In [190]:
## Creating model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [191]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 20, 40)            200000    
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               56400     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [192]:
len(embedded_docs),y.shape

(18285, (18285,))

In [193]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [194]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [195]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 29ms/step - loss: 0.3439 - accuracy: 0.8291 - val_loss: 0.1932 - val_accuracy: 0.9148
Epoch 2/10
128/192 [===================>..........] - ETA: 1s - loss: 0.1510 - accuracy: 0.9412

### Adding Dropout

In [197]:
from tensorflow.keras.layers import Dropout

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

### Performance Metrics And Accuracy¶

In [198]:
y_pred=model.predict_classes(X_test)

In [199]:
from sklearn.metrics import confusion_matrix

In [200]:
confusion_matrix(y_test,y_pred)

array([[ 473, 2946],
       [1025, 1591]], dtype=int64)

In [201]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.3420049710024855